In [5]:
import pandas as pd
import numpy as np
import jupyter_dash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from plotly import graph_objs as go
from dash.dependencies import Input, Output

<ipython-input-5-d8fc588fcbaf>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-5-d8fc588fcbaf>:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [6]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)

In [7]:
#What proportion of trees are in good, fair, or poor health according to the ‘health’ variable ?

In [8]:
trees['spc_common'] = trees['spc_common'].fillna('Missing')
trees['health'] = trees['health'].fillna('Missing')

In [9]:
q_1_health = pd.DataFrame(trees.groupby(['boroname','spc_common','health'])['tree_id'].count())
q_1_health

tree_id
boroname      spc_common           health         
Bronx         American elm         Good          2
                                   Poor          2
              Atlantic white cedar Good          1
              Callery pear         Good          7
              Douglas-fir          Good          1
...                                            ...
Staten Island sweetgum             Good         12
              sycamore maple       Good          1
              white oak            Fair          1
                                   Good          1
              willow oak           Good          1

[184 rows x 1 columns]

In [10]:
q_1_health_prop = q_1_health.div(q_1_health.sum(level=[0,1]))

In [11]:
q_1_health_prop_un = q_1_health_prop.unstack(level=1)

In [12]:
q_1_health_prop_un

tree_id                             \
spc_common            American elm American linden Amur maple   
boroname      health                                            
Bronx         Fair             NaN             NaN        NaN   
              Good        0.500000             NaN        NaN   
              Missing          NaN             NaN        NaN   
              Poor        0.500000             NaN        NaN   
Brooklyn      Fair             NaN        0.100000        NaN   
              Good             NaN        0.850000        1.0   
              Missing          NaN             NaN        NaN   
              Poor             NaN        0.050000        NaN   
Manhattan     Fair        0.666667        0.181818        NaN   
              Good        0.333333        0.818182        NaN   
              Missing          NaN             NaN        NaN   
              Poor             NaN             NaN        NaN   
Queens        Fair        0.500000             NaN        0.5   
              Good        0.500000        0.500000        0.5   
              Missing          NaN             NaN        NaN   
              Poor             NaN        0.500000        NaN   
Staten Island Fair             NaN             NaN        1.0   
              Good             NaN             NaN        NaN   
              Missing          NaN             NaN        NaN   

                                                                            \
spc_common            Atlantic white cedar Callery pear Chinese fringetree   
boroname      health                                                         
Bronx         Fair                     NaN          NaN                NaN   
              Good                     1.0     1.000000                NaN   
              Missing                  NaN          NaN                NaN   
              Poor                     NaN          NaN                NaN   
Brooklyn      Fair                     NaN          NaN                NaN   
              Good                     NaN     1.000000                NaN   
              Missing                  NaN          NaN                NaN   
              Poor                     NaN          NaN                NaN   
Manhattan     Fair                     NaN     0.363636                NaN   
              Good                     NaN     0.636364                NaN   
              Missing                  NaN          NaN                NaN   
              Poor                     NaN          NaN                NaN   
Queens        Fair                     NaN     0.142857                NaN   
              Good                     NaN     0.857143                1.0   
              Missing                  NaN          NaN                NaN   
              Poor                     NaN          NaN                NaN   
Staten Island Fair                     NaN     0.166667                NaN   
              Good                     NaN     0.833333                NaN   
              Missing                  NaN          NaN                NaN   

                                                                        \
spc_common            Douglas-fir Japanese zelkova Kentucky yellowwood   
boroname      health                                                     
Bronx         Fair            NaN              NaN                 NaN   
              Good            1.0         1.000000                 1.0   
              Missing         NaN              NaN                 NaN   
              Poor            NaN              NaN                 NaN   
Brooklyn      Fair            NaN              NaN                 NaN   
              Good            NaN         0.900000                 NaN   
              Missing         NaN              NaN                 NaN   
              Poor            NaN         0.100000                 NaN   
Manhattan     Fair            NaN         0.285714               

In [13]:
#fig = px.bar(q_1_health_prop_un, x="boroname", y="health", color="spc_common", barmode="group")
#pd.options.plotting.backend = "plotly"
#fig = q_1_health_prop_un.plot(kind='bar')
#fig.update_layout(barmode='group')
#fig.show()
temp_df = q_1_health_prop_un.reset_index().melt(id_vars=['boroname','health'])
temp_df = temp_df.fillna(0)
temp_df = temp_df[temp_df['health']!='Missing']

In [14]:
#fig = px.bar(temp_df,x='spc_common',y='value',color='boroname')
#fig

In [15]:
health = temp_df.health.unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="dropdown",
        options=[{"label": x, "value": x} for x in health],
        value=health[0],
        clearable=False,
    ),
    dcc.Graph(id="bar-chart"),
])

@app.callback(
    Output("bar-chart", "figure"), 
    [Input("dropdown", "value")])
def update_bar_chart(health):
    mask = temp_df["health"] == health
    fig = px.bar(temp_df[mask],x='spc_common',y='value',color='boroname',barmode='group') 
    fig.update_layout(title_text='Species Health Ratio Across Burroughs')
    return fig

app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Oct/2021 21:32:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:32:06] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:32:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:32:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:32:06] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:32:06] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:32:06] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:32:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:32:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:32:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2021 21:32:18] "POST /_dash-update-component HTTP/1.1" 

In [16]:
temp_df

,boroname,health,NaN,spc_common,value
0,Bronx,Fair,tree_id,American elm,0.0
1,Bronx,Good,tree_id,American elm,0.5
3,Bronx,Poor,tree_id,American elm,0.5
4,Brooklyn,Fair,tree_id,American elm,0.0
5,Brooklyn,Good,tree_id,American elm,0.0
...,...,...,...,...,...
753,Queens,Fair,tree_id,willow oak,0.0
754,Queens,Good,tree_id,willow oak,1.0
756,Queens,Poor,tree_id,willow oak,0.0
757,Staten Island,Fair,tree_id,willow oak,0.0
